In [1]:
import pandas as pd

In [16]:
e=pd.read_csv('Resources/global_emissions.csv')
e

,Year,Total,Gas Fuel,Liquid Fuel,Solid Fuel,Cement,Gas Flaring,Per Capita
0,1751,3,0,0,3,0,0,NaN
1,1752,3,0,0,3,0,0,NaN
2,1753,3,0,0,3,0,0,NaN
3,1754,3,0,0,3,0,0,NaN
4,1755,3,0,0,3,0,0,NaN
...,...,...,...,...,...,...,...,...
255,2006,8370,1525,3089,3339,356,61,1.27
256,2007,8566,1572,3081,3464,382,68,1.28
257,2008,8783,1631,3122,3571,388,71,1.30
258,2009,8740,1585,3056,3620,413,66,1.28


In [3]:
co2=pd.read_csv('Resources/co2_concentration.csv')
co2=co2.loc[co2['Year']> 1750]

In [4]:
co2=co2.loc[co2['Year']<2011]
co2=co2.rename(columns={'CO2 concentrations (NOAA, 2018)':'concentration'})
co2

,Year,concentration
1722,1752,277.23
1723,1763,277.16
1724,1764,276.40
1725,1773,277.86
1726,1779,280.24
...,...,...
1858,2006,381.90
1859,2007,383.79
1860,2008,385.60
1861,2009,387.43


In [5]:
y=co2.Year.to_list()

In [26]:
def dropper(df, y):
    for x in range(len(df.index)):
#         x=+1
        if df.Year[x] not in y:
            print(x)
            df=df.drop([x])
    return df

In [27]:
df=dropper(e,y)

0
2
3
4
5
6
7
8
9
10
11
14
15
16
17
18
19
20
21
23
24
25
26
27
30
31
32
33
34
35
36
37
38
39
40
41
42
44
46
47
49
50
51
52
53
54
55
56
57
58
59
60
61
62
64
65
66
67
68
69
70
71
72
73
75
77
78
79
80
81
83
84
85
87
88
89
91
97
98
101
102
105
107
109
110
114
115
118
120
121
124
125
127
128
130
131
134
137
140
144
145
146
147
150
152
156
157
161
164
166
170
171
173
175
176
179
192
200
202


In [28]:
df

,Year,Total,Gas Fuel,Liquid Fuel,Solid Fuel,Cement,Gas Flaring,Per Capita
1,1752,3,0,0,3,0,0,NaN
12,1763,3,0,0,3,0,0,NaN
13,1764,3,0,0,3,0,0,NaN
22,1773,4,0,0,4,0,0,NaN
28,1779,4,0,0,4,0,0,NaN
...,...,...,...,...,...,...,...,...
255,2006,8370,1525,3089,3339,356,61,1.27
256,2007,8566,1572,3081,3464,382,68,1.28
257,2008,8783,1631,3122,3571,388,71,1.30
258,2009,8740,1585,3056,3620,413,66,1.28


In [30]:
concentration=co2.concentration.to_list()

In [34]:
df['concentration']=concentration

In [35]:
df

,Year,Total,Gas Fuel,Liquid Fuel,Solid Fuel,Cement,Gas Flaring,Per Capita,concentration
1,1752,3,0,0,3,0,0,NaN,277.23
12,1763,3,0,0,3,0,0,NaN,277.16
13,1764,3,0,0,3,0,0,NaN,276.40
22,1773,4,0,0,4,0,0,NaN,277.86
28,1779,4,0,0,4,0,0,NaN,280.24
...,...,...,...,...,...,...,...,...,...
255,2006,8370,1525,3089,3339,356,61,1.27,381.90
256,2007,8566,1572,3081,3464,382,68,1.28,383.79
257,2008,8783,1631,3122,3571,388,71,1.30,385.60
258,2009,8740,1585,3056,3620,413,66,1.28,387.43


In [36]:
df.to_csv('Resources/model_3.csv')

In [37]:
from DATAcsv_TO_json import converter

In [60]:
import requests
import pandas as pd
import json
import csv
from geojson import Feature, FeatureCollection, Point

In [61]:
csv_filepath='Resources/co2_concentration.csv'
pred_filepath='Resources/chartProjections.csv'
output_path='Resources/CO2_Chart_master.json'
date_col="Year"
y_col= "CO2_concentrations"

In [62]:
def csv_converter(csv_filepath, pred_filepath, output_path, date_col, y_col):

    data_df = pd.read_csv(csv_filepath)
    pred_df = pd.read_csv(pred_filepath)
    
    features = []

    for row in data_df.iterrows():
        features.append(
            {
                "year": f'{int(row[1][date_col])}-01-01 00:00',
                "observed": row[1][y_col]
            }
        )

    for row in pred_df.iterrows():    
        features.append(
            {
                "year": f'{int(row[1][date_col])}-01-01 00:00',
                "easing": row[1]["easing"],
                "maintaining": row[1]["maintaining"],
                "increasing": row[1]["increase"] 
            }
        )
    with open(output_path, "w") as f:
        f.write("%s" % features)


In [63]:
# data formatting for diverging chart type
csv_converter(csv_filepath, pred_filepath, output_path, date_col, y_col)

In [65]:
df=pd.read_csv('Resources/food_lifecycle_emissions.csv')
df
del df["Unnamed: 8"]
df

,Food product,Land use change,Animal Feed,Farm,Processing,Transport,Packging,Retail
0,Wheat & Rye (Bread),0.1,0.0,0.8,0.2,0.1,0.1,0.1
1,Maize (Meal),0.3,0.0,0.5,0.1,0.1,0.1,0.0
2,Barley (Beer),0.0,0.0,0.2,0.1,0.0,0.5,0.3
3,Oatmeal,0.0,0.0,1.4,0.0,0.1,0.1,0.0
4,Rice,0.0,0.0,3.6,0.1,0.1,0.1,0.1
5,Potatoes,0.0,0.0,0.2,0.0,0.1,0.0,0.0
6,Cassava,0.6,0.0,0.2,0.0,0.1,0.0,0.0
7,Cane Sugar,1.2,0.0,0.5,0.0,0.8,0.1,0.0
8,Beet Sugar,0.0,0.0,0.5,0.2,0.6,0.1,0.0
9,Other Pulses,0.0,0.0,1.1,0.0,0.1,0.4,0.0


In [73]:
land=sum(df["Land use change"].to_list())
feed=sum(df["Animal Feed"].to_list())
farm=sum(df["Farm"].to_list())
processing=sum(df["Processing"].to_list())
transport=sum(df["Transport"].to_list())
packaging=sum(df["Packging"].to_list())
retail=sum(df["Retail"].to_list())


In [74]:
land

54.20000000000001

In [88]:
df={
    'land':int(land),
    'feed':int(feed),
    'farm':int(farm),
    'processing':int(processing),
    'transport':int(transport),
    'packging':int(packaging),
    'retail':int(retail)
}
df=[df]

In [92]:
total=pd.DataFrame(df)
total=total.T


In [97]:
total

,0
land,54
feed,19
farm,149
processing,10
transport,8
packging,11
retail,3


In [109]:
total=total.rename(columns={'0':'value'})
total['color']=["#FEF5A8","#CF519A","#FFD8A0","#F5DDB7","#B9D0E8","#D6BFD4","#D3D7CF"]


In [105]:
total=total.rename(columns={0:'value'})
total=total.reset_index()


In [107]:
total=total.rename(columns={'index':'name'})
total

,name,value,color
0,land,54,"am4core.color(""#FEF5A8"")"
1,feed,19,"am4core.color(""#CF519A"")"
2,farm,149,"am4core.color(""#FFD8A0"")"
3,processing,10,"am4core.color(""#F5DDB7"")"
4,transport,8,"am4core.color(""#B9D0E8"")"
5,packging,11,"am4core.color(""#D6BFD4"")"
6,retail,3,"am4core.color(""#D3D7CF"")"


In [110]:
total

,name,value,color
0,land,54,#FEF5A8
1,feed,19,#CF519A
2,farm,149,#FFD8A0
3,processing,10,#F5DDB7
4,transport,8,#B9D0E8
5,packging,11,#D6BFD4
6,retail,3,#D3D7CF


In [111]:
total.to_csv('Resources/footChart.csv', index=False)